ConvLSTM trained on gridded forcings for all stations

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn, utils
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, conv_lstm, datasets
import torch.autograd as autograd
import pickle

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190720-131601'

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../log.out', mode='a')
chandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - {} - %(message)s'.format(time_stamp))
fhandler.setFormatter(formatter)
chandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.INFO)

In [3]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
logger.warning('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)

CUDA Available
2019-07-20 13:16:02,045 - 20190720-131601 - cuda devices: ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']


In [4]:
seq_len = 50
seq_steps = 1

train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(hours=seq_len * seq_steps)  # first day for which to make a prediction in train set
train_end = '2012-09-30'
val_start = '2012-10-01'
val_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'

In [5]:
rdrs_vars = [4,5]
train_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, train_start, train_end)
val_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, val_start, val_end,
                                   conv_scalers=train_dataset.conv_scalers, fc_scalers=train_dataset.fc_scalers)
test_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, test_start, test_end, 
                                    conv_scalers=train_dataset.conv_scalers, fc_scalers=train_dataset.fc_scalers)

../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]
../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]
../src/load_data.py:43: RuntimeWarning: invalid value encountered in greater
  rdrs_data[:,i,:,:] = rdrs_nc[forcing_variables[i]][:]


In [6]:
# Train model
num_epochs = 200
learning_rate = 2e-3
patience = 100
min_improvement = 1.0
best_loss_model = (-1, np.inf, None)

# Prepare model
stateful_lstm = True
lstm_layers = 3
conv_hidden_dims = [20] * (lstm_layers - 1) + [1]
H_fc = 8
batch_size = 45
fc_layers = 2
kernel_size = (3,3)
dropout = 0.0
pooling = [True] * lstm_layers
model = conv_lstm.ConvLSTMRegression((train_dataset.conv_height, train_dataset.conv_width), train_dataset.n_fc_vars, train_dataset.n_conv_vars, conv_hidden_dims, 
                                     kernel_size, lstm_layers, dropout, fc_layers, H_fc, pooling, batch_first=True).to(device)
model = torch.nn.DataParallel(model, device_ids=list(range(num_devices)))
loss_fn = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

writer = SummaryWriter()
param_description = {'H_conv': conv_hidden_dims, 'H_fc': H_fc, 'batch_size': batch_size, 'lstm_layers': lstm_layers, 'fc_layers': fc_layers, 'kernel_size': kernel_size, 'loss': loss_fn, 
                     'optimizer': optimizer, 'lr': learning_rate, 'patience': patience, 'min_improvement': min_improvement, 'pooling': pooling, 'stateful_lstm': stateful_lstm,
                     'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 'dropout': dropout, 'train_start': train_start, 'train_end': train_end, 'val_start': val_start, 
                     'val_end': val_end, 'test_start': test_start, 'test_end': test_end, 'n_conv_vars': train_dataset.n_conv_vars, 'n_fc_vars': train_dataset.n_fc_vars,
                     'train len':len(train_dataset), 'val len': len(val_dataset), 'conv_height': train_dataset.conv_height, 'conv_width': train_dataset.conv_width, 'test len': len(test_dataset)}
writer.add_text('Parameter Description', str(param_description))

In [7]:
if stateful_lstm:
    train_sampler = datasets.StatefulBatchSampler(train_dataset, batch_size)
    val_sampler = datasets.StatefulBatchSampler(val_dataset, batch_size)
    test_sampler = datasets.StatefulBatchSampler(test_dataset, batch_size)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_sampler, pin_memory=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_sampler=val_sampler, pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_sampler, pin_memory=True)
else:
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=False, pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True)

In [8]:
for epoch in range(num_epochs):
    model.train()

    epoch_losses = torch.tensor(0.0)
    conv_hidden_states = None
    for i, train_batch in enumerate(train_dataloader):
        y_train = train_batch['y'].reshape((-1, 1)).to(device, non_blocking=True)
        if not stateful_lstm:
            conv_hidden_states = None
        
        y_pred, conv_hidden_states = model(train_batch['x_conv'], train_batch['x_fc'], conv_hidden_states=conv_hidden_states)
        loss = loss_fn(y_pred, y_train)
        epoch_losses += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    epoch_loss = (epoch_losses / len(train_dataloader)).item()
    print('Epoch', epoch, 'mean train loss:\t{}'.format(epoch_loss))
    writer.add_scalar('loss', epoch_loss, epoch)
    
    # eval on validation split
    model.eval()
    val_pred = []
    val_y = []
    for i, val_batch in enumerate(val_dataloader):
        if not stateful_lstm:
            conv_hidden_states = None
        
        batch_pred, conv_hidden_states = model(val_batch['x_conv'], val_batch['x_fc'], conv_hidden_states=conv_hidden_states)
        val_pred.append(batch_pred.detach().reshape(-1))
        val_y.append(val_batch['y'])

    val_pred = torch.cat(val_pred).cpu()
    val_y = torch.cat(val_y)
    val_nse, val_mse = evaluate.evaluate_daily('All Stations', pd.Series(val_pred), pd.Series(val_y))
    print('Epoch {} mean val mse:    \t{},\tnse: {}'.format(epoch, val_mse, val_nse))
    writer.add_scalar('loss_eval', val_mse, epoch)

    if val_mse < best_loss_model[1] - min_improvement:
        best_loss_model = (epoch, val_mse, model.state_dict())  # new best model
    elif epoch > best_loss_model[0] + patience:
        print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
        break

print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
model.load_state_dict(best_loss_model[2])
load_data.pickle_model('ConvLSTM+LinearLayer_VIC', model, 'allStations', time_stamp)

Epoch 0 mean train loss:	3327.846435546875
Epoch 0 mean val mse:    	674.3103637695312,	nse: -0.0670194804191575
Epoch 1 mean train loss:	3341.44677734375
Epoch 1 mean val mse:    	671.3687133789062,	nse: -0.062364530018717
Epoch 2 mean train loss:	3339.314697265625
Epoch 2 mean val mse:    	670.06884765625,	nse: -0.06030769452459506
Epoch 3 mean train loss:	3347.297119140625
Epoch 3 mean val mse:    	665.6358032226562,	nse: -0.05329291377670531
Epoch 4 mean train loss:	3343.857421875
Epoch 4 mean val mse:    	663.3946533203125,	nse: -0.049746596008506616
Epoch 5 mean train loss:	3331.27880859375
Epoch 5 mean val mse:    	666.1381225585938,	nse: -0.05408774698527008
Epoch 6 mean train loss:	3317.00634765625
Epoch 6 mean val mse:    	666.0848388671875,	nse: -0.05400352433808653
Epoch 7 mean train loss:	3300.714599609375
Epoch 7 mean val mse:    	662.960693359375,	nse: -0.04905996635345544
Epoch 8 mean train loss:	3282.926025390625
Epoch 8 mean val mse:    	660.8109741210938,	nse: -0.045

In [9]:
logger.warning('predicting')
model.eval()

predict = test_dataset.data_runoff.copy()
predict['runoff'] = np.nan
pred_array = []
for i, test_batch in enumerate(test_dataloader):
    if not stateful_lstm:
        conv_hidden_states = None
    pred, conv_hidden_states = model(test_batch['x_conv'], test_batch['x_fc'], conv_hidden_states=conv_hidden_states)
    pred_array.append(pred.detach().reshape(-1))
    
pred_array = torch.cat(pred_array).cpu()

if stateful_lstm:
    # reorder time series
    pred_indices = np.array(list(test_sampler.__iter__())).reshape(-1)
    pred_array = pred_array[pred_indices.argsort()]

if len(pred_array) != len(predict):
    print('Warning: length of prediction {} and actuals {} does not match. Ignoring excess actuals.'.format(len(pred_array), len(predict)))
    predict = predict.iloc[:len(pred_array)]
    
predict['runoff'] = pred_array

2019-07-21 08:09:01,710 - 20190720-131601 - predicting


In [10]:
actuals = test_dataset.data_runoff.copy()
if len(actuals) != len(predict):
    print('Warning: length of prediction {} and actuals {} does not match. Ignoring excess actuals.'.format(len(predict), len(actuals)))
    actuals = actuals[:len(predict)]

nse_list = []
mse_list = []
for station in predict['station'].unique():
    nse, mse = evaluate.evaluate_daily(station, predict[predict['station'] == station]['runoff'], actuals[actuals['station'] == station]['runoff'], writer=writer)
    nse_list.append(nse)
    mse_list.append(mse)
    
    print(station, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')

print('Median NSE (clipped to 0)', np.median(nse_list), '/ Min', np.min(nse_list), '/ Max', np.max(nse_list))
print('Median MSE (clipped to 0)', np.median(mse_list), '/ Min', np.min(mse_list), '/ Max', np.max(mse_list))

02GA010 	NSE: -0.07388365097904503 	MSE: 511.99931870529247 (clipped to 0)
02GA018 	NSE: -0.002476636847838831 	MSE: 251.77519572332113 (clipped to 0)
02GA038 	NSE: -0.0797873901592312 	MSE: 177.66655098814144 (clipped to 0)
02GA047 	NSE: -0.014628932437123376 	MSE: 79.53678703922041 (clipped to 0)
02GB001 	NSE: -0.7792550976466956 	MSE: 13361.98372980398 (clipped to 0)
02GB007 	NSE: -1.0436004539457264 	MSE: 63.90750308669408 (clipped to 0)
02GC002 	NSE: -0.18334082723786893 	MSE: 152.062887290391 (clipped to 0)
02GC007 	NSE: -0.049852194645182424 	MSE: 31.622959601494074 (clipped to 0)
02GC010 	NSE: -0.3478479344235088 	MSE: 80.12945160254745 (clipped to 0)
02GC018 	NSE: -0.5111983072287429 	MSE: 102.34862811948055 (clipped to 0)
02GC026 	NSE: -0.010097236136441756 	MSE: 162.45671963056435 (clipped to 0)
02GD004 	NSE: -0.4922304108048057 	MSE: 82.16679265921161 (clipped to 0)
02GE007 	NSE: -1.4000063749496219 	MSE: 86.8203709652838 (clipped to 0)
02GG002 	NSE: -0.00011499010298354762

In [11]:
writer.close()

In [12]:
save_df = pd.merge(predict.rename({'runoff': 'prediction'}, axis=1), actuals.rename({'runoff': 'actual'}, axis=1), 
                   on=['date', 'station'])[['date', 'station', 'prediction', 'actual']]
load_data.pickle_results('ConvLSTM+LinearLayer_VIC', save_df, time_stamp)

'ConvLSTM+LinearLayer_VIC_20190720-131601.pkl'

In [13]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190721-081223'